In [3]:
import subprocess
import sys

def install_and_import(package_name, import_name=None):
    import_name = import_name or package_name
    try:
        __import__(import_name)
    except ModuleNotFoundError:
        print(f"{package_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installed successfully.")

# Modules to install
install_and_import("python-dotenv", "dotenv")
install_and_import("numpy")
install_and_import("pvporcupine")
install_and_import("pvcobra")
install_and_import("whisper")
install_and_import("pvrecorder")
install_and_import("torch")
install_and_import("matplotlib")
install_and_import("librosa")

import time
from collections import deque
import os
from dotenv import load_dotenv
import numpy as np
import pvporcupine
import pvcobra
import whisper
from pvrecorder import PvRecorder
import torch


load_dotenv()

porcupine = pvporcupine.create(
    access_key=os.environ.get("ACCESS_KEY"),
    keyword_paths=[os.environ.get("WAKE_WORD_MODEL_PATH")],
)

cobra = pvcobra.create(
    access_key=os.environ.get("ACCESS_KEY"),
)

recoder = PvRecorder(device_index=-1, frame_length=512)

# frame length = 512
# samples per frame = 16,000
# 1 sec = 16,000 / 512


class Transcriber:
    def __init__(self, model) -> None:
        print("loading model")
        # TODO: put model on GPU
        self.model = whisper.load_model(model)
        print("loading model finished")
        self.prompts = os.environ.get("WHISPER_INITIAL_PROMPT", "")
        print(f"Using prompts: {self.prompts}")

    def transcribe(self, frames):
        transcribe_start = time.time()
        samples = np.array(frames, np.int16).flatten().astype(np.float32) / 32768.0

        # audio = whisper.pad_or_trim(samples)
        # print(f"{transcribe_start} transcribing {len(frames)} frames.")
        # # audio = whisper.pad_or_trim(frames)

        # # make log-Mel spectrogram and move to the same device as the model
        # mel = whisper.log_mel_spectrogram(audio).to(self.model.device)

        # # decode the audio
        # options = whisper.DecodingOptions(fp16=False, language="english")
        # result = whisper.decode(self.model, mel, options)

        result = self.model.transcribe(
            audio=samples,
            language="en",
            fp16=False,
            initial_prompt=self.prompts,
        )

        # print the recognized text
        transcribe_end = time.time()
        # print(
        #     f"{transcribe_end} - {transcribe_end - transcribe_start}sec: {result.get('text')}",
        #     flush=True,
        # )
        return result.get("text", "speech not detected")


transcriber = Transcriber(os.environ.get("WHISPER_MODEL"))

sample_rate = 16000
frame_size = 512
vad_mean_probability_sensitivity = float(os.environ.get("VAD_SENSITIVITY"))

try:
    recoder.start()

    max_window_in_secs = 3
    window_size = sample_rate * max_window_in_secs
    samples = deque(maxlen=(window_size * 6))
    vad_samples = deque(maxlen=25)
    is_recording = False

    while True:
        data = recoder.read()
        vad_prob = cobra.process(data)
        vad_samples.append(vad_prob)
        # print(f"{vad_prob} - {np.mean(vad_samples)} - {len(vad_samples)}")
        if porcupine.process(data) >= 0:
            print(f"Detected wakeword")
            is_recording = True
            samples.clear()

        if is_recording:
            if (
                len(samples) < window_size
                or np.mean(vad_samples) >= vad_mean_probability_sensitivity
            ):
                samples.extend(data)
                # print(f"listening - samples: {len(samples)}")
            else:
                print("is_recording: False")
                print(transcriber.transcribe(samples))
                is_recording = False
except KeyboardInterrupt:
    recoder.stop()
finally:
    porcupine.delete()
    recoder.delete()
    cobra.delete()


pvporcupine not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.8 MB/s eta 0:00:00
pvporcupine installed successfully.
pvcobra not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.1 MB/s eta 0:00:00
pvcobra installed successfully.
whisper not found. Installing...
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41197 sha256=ea6fee86bdac7adb6dfae83810b2c6f0a7dba8ef97283430761d737f0c040b6f
  Stored in directory: /Users/charissaluk/Library/Caches/pip/wheels/34/b8/4e/9c4c3351d670e06746a340fb4b7d854c76517eec225e5b32b1
Successfully built whisper
whisper installed successfully.
pvrecorder not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 13.4 MB/s eta 0:00:00a 0:00:01
pvrecorder installed successfully.
torch not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

PorcupineInvalidArgumentError: Initialization failed:
  [0] Keyword file (.ppn) file has incorrect format or belongs to a different platform.
  [1] Picovoice Error (code `00000136`)
  [2] Loading keyword file at `models/astra.ppn` failed with `INVALID_ARGUMENT`.

In [25]:
# Identify instrument from transcription

import string

# command = transcriber.transcribe(samples)
command = "Give me forceps"
instruments = ['forceps', 'scalpel', 'scissors', 'tweezers']

instrument = ''
for word in command.split():
    word = (word.translate(str.maketrans('', '', string.punctuation))).lower()
    if word in instruments:
        instrument = word

if instrument == '':
    print("No instrument found")


RuntimeError: Expected 2D or 3D (batch mode) tensor with possibly 0 batch size and other non-zero dimensions for input, but got: [1, 1, 0]

In [5]:
import os
from dotenv import load_dotenv
import pvporcupine

load_dotenv()

porcupine = pvporcupine.create(
    access_key=os.environ["ACCESS_KEY"],
    keyword_paths=[os.environ["WAKE_WORD_MODEL_PATH"]],
)
print("Porcupine loaded successfully ✅")


PorcupineInvalidArgumentError: Initialization failed:
  [0] Keyword file (.ppn) file has incorrect format or belongs to a different platform.
  [1] Picovoice Error (code `00000136`)
  [2] Loading keyword file at `models/astra.ppn` failed with `INVALID_ARGUMENT`.

In [35]:
# Yolo trained on DocCheck (Rona) dataset

from ultralytics import YOLO
import torch

# load yolov5 from online
# TODO: change to local
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/instrument_detector_model.pt', force_reload=True)  # load a custom model

# Predict with the model
results = model("./eval-images/inst-notape.jpg")  # predict on an image


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\maxis/.cache\torch\hub\master.zip
YOLOv5  2025-4-6 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 267 layers, 46167513 parameters, 0 gradients, 107.8 GFLOPs
Adding AutoShape... 
C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [52]:
# Translate results
results.names = {0: 'Standard Anatomical Tweezers',
 1: 'Slim Anatomical Tweezers',
 2: 'Surgical Tweezers',
 3: 'Splinter Tweezers',
 4: 'Scalpel Handle No. 3',
 5: 'Scalpel Handle No. 4',
 6: 'Clenched Scalpel',
 7: 'Narrow Scalpel',
 8: 'Surgical Scissors Sharp/Sharp',
 9: 'Surgical Scissors Sharp/Narrow',
 10: 'Standard Dissecting Scissors',
 11: 'Dissecting Needle'}

# map labels to basic voice commands "forceps", "scalpel", "scissors", "needle"
map_instruments = {results.names[0]: 'forceps',
                   results.names[1]: 'forceps',
                   results.names[2]: 'forceps',
                   results.names[3]: 'forceps',
                   results.names[4]: 'scalpel',
                   results.names[5]: 'scalpel',
                   results.names[6]: 'scalpel',
                   results.names[7]: 'scalpel',
                   results.names[8]: 'scissors',
                   results.names[9]: 'scissors',
                   results.names[10]: 'scissors',
                   results.names[11]: 'needle'}

In [67]:
detections = results.xyxy[0]

conf_threshold = 0.65
# instruments = 'forceps'

# TODO: account for when there's multiple types of the same instrument

x_midpoint = 0
y_midpoint = 0

for *box, conf, cls in detections:
    cls = int(cls)
    x1, y1, x2, y2 = box
    # if detection matches the instrument from voice command
    if map_instruments[results.names[cls]] == instrument and conf >= conf_threshold :
        x_midpoint = (x1 + x2) / 2
        y_midpoint = (y1 + y2) / 2
        print(f"Box center: ({x_midpoint:.0f}, {y_midpoint:.0f}) | Confidence: {conf:.2f} | Class: {results.names[cls]}")
        break

if x_midpoint == 0:
    print("No instrument found")

Box center: (1946, 1560) | Confidence: 0.79 | Class: Surgical Tweezers
